# Qaintellect training example 2: multiple qubits and measurement operators

In this example, we will demonstrate the optimisation of a quantum circuit with multiple qubits and measurement operators.

In [1]:
using Pkg; Pkg.add(url="https://github.com/Qaintum/Qaintessent.jl");
using Qaintellect
using LinearAlgebra
using Flux
using IterTools: ncycle

   Updating git-repo `https://github.com/Qaintum/Qaintessent.jl`


   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


┌ Warning: Some registries failed to update:
│     — /home/ga53vuw/.julia/registries/General — failed to fetch from repo
└ @ Pkg.Types /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/Pkg/src/Types.jl:1194
  Resolving package versions...
No Changes to `/import/home/ga53vuw/Documents/PhD/projects/QAI/Qaintellect.jl/Project.toml`
No Changes to `/import/home/ga53vuw/Documents/PhD/projects/QAI/Qaintellect.jl/Manifest.toml`


We create a parametrized quantum circuit with $N$ qubits. We then optimize these parameters using the Flux framework.

In [2]:
# construct parametrized circuit

N = 4

ry = RyGate(√2)
n = randn(Float64, 3)
n /= norm(n)
rg = RotationGate(0.2π, n)
@show(rg)

cgc = [
    circuit_gate(3, HadamardGate()),
    circuit_gate(2, RzGate(1.5π), (1, 4)), # controlled gate
    circuit_gate(2, 3, SwapGate()),
    circuit_gate(3, PhaseShiftGate(0.3)),
    circuit_gate(3, rg),
    circuit_gate(1, ry),
]
meas = [MeasurementOperator(Matrix{Float64}(I, 2^N, 2^N), Tuple(1:N)), MeasurementOperator(Hermitian(randn(ComplexF64, 2^N, 2^N)), Tuple(1:N))]

c = Circuit{N}(cgc, meas)

rg = RotationGate([-0.3770941769323742, -0.18191239085016125, -0.4684997756909257])



    4 ————————•—————————————————————
              |                     
    3 —[H ]—————————x————[Pϕ]——[Rθ]—
              |     |               
    2 ———————[Rz]———x———————————————
              |                     
    1 ————————•————————————————[Ry]—


In [3]:
# set up model
model(ψ) = dot([0.3, -1.2], c(ψ))

# create loss function
loss(x, y) = Flux.mse(model(x), y)

# gather parameters from Circuit
paras = Flux.params(c)

# freeze parameter ry.θ, equivalent to delete!(paras, Qaintessent.get_trainable(ry))
delete!(paras, ry.θ)

paras

Params([[4.71238898038469], [0.3], [-0.3770941769323742, -0.18191239085016125, -0.4684997756909257], 
  [1 ,  1]  =  1.0+0.0im
  [2 ,  2]  =  1.0+0.0im
  [3 ,  3]  =  1.0+0.0im
  [4 ,  4]  =  1.0+0.0im
  [5 ,  5]  =  1.0+0.0im
  [6 ,  6]  =  1.0+0.0im
  [7 ,  7]  =  1.0+0.0im
  [8 ,  8]  =  1.0+0.0im
  [9 ,  9]  =  1.0+0.0im
  [10, 10]  =  1.0+0.0im
  [11, 11]  =  1.0+0.0im
  [12, 12]  =  1.0+0.0im
  [13, 13]  =  1.0+0.0im
  [14, 14]  =  1.0+0.0im
  [15, 15]  =  1.0+0.0im
  [16, 16]  =  1.0+0.0im, 
  [1 ,  1]  =  0.412232+0.0im
  [2 ,  1]  =  1.12021-0.256048im
  [3 ,  1]  =  0.302475+0.717748im
  [4 ,  1]  =  -2.45391-0.991839im
  [5 ,  1]  =  -0.84334+0.657044im
  [6 ,  1]  =  -0.843339+0.632229im
  [7 ,  1]  =  -1.27159+1.11835im
  [8 ,  1]  =  -0.360019-0.698453im
  [9 ,  1]  =  -1.21463-0.462286im
  [10,  1]  =  0.246136+0.40925im
  [11,  1]  =  0.614472+0.0312881im
  [12,  1]  =  -0.301658+0.415073im
  ⋮
  [4 , 16]  =  -0.282751-1.06899im
  [5 , 16]  =  -1.55817+1.4844im
  [6 , 1

In [4]:
# input quantum state
ψ = randn(ComplexF64, 2^N)
ψ /= norm(ψ)

# desired output
e = 0.65

# set up data for training
data = ncycle([(ψ, e)], 64)

# define optimizer
opt = Descent(1)

# define evaluation function
evalcb() = @show(loss(ψ, e))

println("Initial model evaluation: $(model(ψ)), target: $e")

Flux.train!(loss, paras, data, opt, cb=Flux.throttle(evalcb, 0.01))

println("Final model evaluation: $(model(ψ)), target: $e")

Initial model evaluation: -0.6411689764188402, target: 0.65
loss(ψ, e) = 3.2547793683377075
loss(ψ, e) = 7.703719777548943e-30
Final model evaluation: 0.6500000000000004, target: 0.65


In [5]:
# check: Ry gate has not changed (frozen parameter)
@show(ry)

# on the other hand, parameters of general rotation gate are now different
@show(rg);

ry = RyGate([1.4142135623730951])
rg = RotationGate([2.2792445765601532, 14.620422188462491, 4.557172204234055])
